<a href="https://colab.research.google.com/github/wenchun0731/github-colab/blob/main/%E5%BD%B1%E7%89%87%E5%88%87%E6%88%90%E5%BD%B1%E6%A0%BC%2B%E8%BB%8A%E7%89%8C%E6%88%AA%E5%8F%96.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#掛載雲端硬碟
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
#切成影格

import cv2
#設定擷取方法
def extract_frames(video_path, output_folder, frame_interval):
    # 打開影片檔案
    cap = cv2.VideoCapture(video_path)

    # 確認影片檔案是否成功打開
    if not cap.isOpened():
        print("Error: Unable to open video file.")
        return

    # 計數器，用來計算擷取的影格數量
    frame_count = 0

    # 逐一讀取影格
    while True:
        ret, frame = cap.read()

        # 確認是否成功讀取影格
        if not ret:
            break

        # 每隔 frame_interval 幀擷取一次影格
        if frame_count % frame_interval == 0:
            # 影格儲存的檔案名稱
            frame_filename = f"{output_folder}/frame_{frame_count:04d}.jpg"

            # 儲存影格
            cv2.imwrite(frame_filename, frame)

        # 更新計數器
        frame_count += 1

    # 釋放資源
    cap.release()

    print(f"Total frames extracted: {frame_count}")

#只需改以下

# 影片檔案路徑(輸入)
video_path = "/content/drive/MyDrive/Finetune/Licence_Finetune/test_video/test_video3_60fps_clip.mp4"

# 輸出資料夾
output_folder = "/content/drive/MyDrive/Finetune/Licence_Finetune/FrameOutput"

# 執行擷取影格的函式(輸入影片,輸出資料夾,每隔?幀擷取一次影格)
extract_frames(video_path, output_folder, frame_interval=1)


In [ ]:
#照片偵測，產出座標檔(模型之後改V8,來源改影格輸出)
!python /content/yolov7/detect.py --weights /content/drive/MyDrive/Finetune/Licence_Finetune/train/exp2/weights/best.pt --conf-thres=0.05 --iou-thres=0.5 --img-size 640 --source /content/drive/MyDrive/Finetune/Licence_Finetune/FrameOutput --project /content/drive/MyDrive/Finetune/Licence_Finetune/detect --save-txt --save-conf

In [2]:
# 安裝Pillow庫
!pip install Pillow

# 從Pillow庫中導入Image
from PIL import Image
import os

In [4]:
#擷取車牌

# 定義標籤文件和圖像文件的目錄
image_dir = '/content/drive/MyDrive/car'  # 輸入圖像的目錄(這裡指定為影格的輸出)
label_dir = '/content/drive/MyDrive/Finetune/Licence_Finetune/detect/exp7/labels'  # 輸入圖片座標的目錄(會一直改)
output_dir = '/content/drive/MyDrive/Finetune/Licence_Finetune/coordinate_ouput'  # 輸出目錄


# 確保保存截取圖像的目錄存在
os.makedirs(output_dir, exist_ok=True)

# 獲取所有標籤文件
label_files = [f for f in os.listdir(label_dir) if f.endswith('.txt')]

# 定義拓寬的像素值
padding = 10  # 可以根據需要調整拓寬的像素值

# 遍歷每個標籤文件並進行處理
for label_file in label_files:
    label_file_path = os.path.join(label_dir, label_file)

    # 讀取標籤文件內容
    with open(label_file_path, 'r') as file:
        content = file.read().strip()

        # 分割數字
        numbers = list(map(float, content.split()))

        # 進行計算
        if len(numbers) >= 6:
            # 對應的圖像文件名
            image_file = label_file.replace('.txt', '.jpg')
            image_path = os.path.join(image_dir, image_file)

            # 確保圖像文件存在
            if os.path.exists(image_path):
                # 打開圖像並獲取其尺寸
                image = Image.open(image_path)
                width, height = image.size

                center_x = numbers[1] * width
                center_y = numbers[2] * height
                box_width = numbers[3] * width
                box_height = numbers[4] * height

                # 計算左上角和右下角的座標
                left = int(center_x - box_width / 2)
                top = int(center_y - box_height / 2)
                right = int(center_x + box_width / 2)
                bottom = int(center_y + box_height / 2)

                # 拓寬邊界
                left = max(0, left - padding)
                top = max(0, top - padding)
                right = min(width, right + padding)
                bottom = min(height, bottom + padding)

                # 截取圖像
                cropped_image = image.crop((left, top, right, bottom))

                # 保存截取的圖像
                cropped_image_file = f'cropped_{os.path.splitext(image_file)[0]}.jpg'
                cropped_image_path = os.path.join(output_dir, cropped_image_file)
                cropped_image.save(cropped_image_path)
                print(f"截取的圖像已保存到: {cropped_image_path}")
            else:
                print(f"對應的圖像文件不存在: {image_path}")
        else:
            print(f"文件內容格式錯誤，數字數量不足: {label_file_path}")


截取的圖像已保存到: /content/drive/MyDrive/Finetune/Licence_Finetune/coordinate_ouput/cropped_IMG_6201.jpg
截取的圖像已保存到: /content/drive/MyDrive/Finetune/Licence_Finetune/coordinate_ouput/cropped_IMG_6200.jpg
截取的圖像已保存到: /content/drive/MyDrive/Finetune/Licence_Finetune/coordinate_ouput/cropped_IMG_6202.jpg
截取的圖像已保存到: /content/drive/MyDrive/Finetune/Licence_Finetune/coordinate_ouput/cropped_IMG_6204.jpg
截取的圖像已保存到: /content/drive/MyDrive/Finetune/Licence_Finetune/coordinate_ouput/cropped_IMG_6205.jpg
截取的圖像已保存到: /content/drive/MyDrive/Finetune/Licence_Finetune/coordinate_ouput/cropped_IMG_6206.jpg
截取的圖像已保存到: /content/drive/MyDrive/Finetune/Licence_Finetune/coordinate_ouput/cropped_IMG_6207.jpg
截取的圖像已保存到: /content/drive/MyDrive/Finetune/Licence_Finetune/coordinate_ouput/cropped_IMG_6210.jpg
截取的圖像已保存到: /content/drive/MyDrive/Finetune/Licence_Finetune/coordinate_ouput/cropped_IMG_6211.jpg
